<a href="https://colab.research.google.com/github/Arombo34/Ai-future-directions/blob/main/Aiprototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# =========================================
# EDGE AI PROTOTYPE: IMAGE CLASSIFICATION
# TRAIN → CONVERT TO TFLITE → TEST
# =========================================

import tensorflow as tf
import numpy as np

# =========================================
# 1. LOAD SAMPLE DATA (CIFAR-10 placeholder)
# =========================================
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.cifar10.load_data()

# Normalize images
x_train = x_train / 255.0
x_val = x_val / 255.0

# One-hot encode labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_val = tf.keras.utils.to_categorical(y_val, 10)

# Class names for CIFAR-10
classes = ['Airplane','Automobile','Bird','Cat','Deer','Dog','Frog','Horse','Ship','Truck']

# =========================================
# 2. BUILD LIGHTWEIGHT CNN MODEL
# =========================================
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(32,32,3)),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# =========================================
# 3. TRAIN MODEL
# =========================================
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=3, batch_size=64)

# Save trained model
model.save("edge_ai_model.h5")
print("✅ Trained model saved as edge_ai_model.h5")

# =========================================
# 4. CONVERT MODEL TO TENSORFLOW LITE
# =========================================
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("edge_ai_model.tflite", "wb") as f:
    f.write(tflite_model)
print("✅ TFLite model saved as edge_ai_model.tflite")

# =========================================
# 5. TEST TFLITE MODEL ON ONE SAMPLE IMAGE
# =========================================
interpreter = tf.lite.Interpreter(model_path="edge_ai_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Pick one test image and cast to float32
test_img = np.expand_dims(x_val[0], axis=0).astype(np.float32)

interpreter.set_tensor(input_details[0]['index'], test_img)
interpreter.invoke()

prediction = interpreter.get_tensor(output_details[0]['index'])
pred_class = classes[np.argmax(prediction)]

print("\nPrediction probabilities:", prediction)
print("Predicted class:", pred_class)


Epoch 1/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 48ms/step - accuracy: 0.3079 - loss: 1.8851 - val_accuracy: 0.4784 - val_loss: 1.4456
Epoch 2/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 38s 49ms/step - accuracy: 0.5080 - loss: 1.3623 - val_accuracy: 0.5504 - val_loss: 1.2489
Epoch 3/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 37s 48ms/step - accuracy: 0.5710 - loss: 1.2127 - val_accuracy: 0.5768 - val_loss: 1.2028


✅ Trained model saved as edge_ai_model.h5
Saved artifact at '/tmp/tmpvcqft5mv'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor_10')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  138499069653840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138499073926800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138499069645392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138499069646928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138498927050832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138498927051408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138498927052560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138498927051216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138498927052944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138498927052368: TensorSpec(shape=(), dt

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
